In [444]:
import numpy as np

In [445]:
def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [446]:
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, a_prev, c_prev, parameters):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, parameters)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the memory value
    """

    # Retrieve parameters from "parameters"
    Wf = parameters["Wf"]
    bf = parameters["bf"]
    Wi = parameters["Wi"]
    bi = parameters["bi"]
    Wc = parameters["Wc"]
    bc = parameters["bc"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    Wy = parameters["Wy"]
    by = parameters["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    n_x, m = xt.shape
    n_y, n_a = Wy.shape

    ### START CODE HERE ###
    # Concatenate a_prev and xt (≈3 lines)
    concat = np.zeros((n_a + n_x, m))
    concat[: n_a, :] = a_prev
    concat[n_a :, :] = xt

#     print("concat", concat)

    # Compute values for ft, it, cct, c_next, ot, a_next using the formulas given figure (4) (≈6 lines)
    ft = sigmoid(np.dot(Wf, concat) + bf)
    # print("ft = ",ft)
    it = sigmoid(np.dot(Wi, concat) + bi)
    cct = np.tanh(np.dot(Wc, concat) + bc)
    # print("c_prev",c_prev)
    # print("ft = ", ft)
    # print("c_prev = ", c_prev)
    # print("ft * c_prev = ", ft*c_prev)
    c_next = ft * c_prev + it * cct
    # print(c_next)
    ot = sigmoid(np.dot(Wo, concat) + bo)
    a_next = ot * np.tanh(c_next)
    
    # Compute prediction of the LSTM cell (≈1 line)
    yt_pred = softmax(np.dot(Wy, a_next) + by)
    # print("yt_pred = ",yt_pred)
    ### END CODE HERE ###

    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, parameters)

    return a_next, c_next, yt_pred, cache

In [447]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
c_prev = np.random.randn(5,10)
Wf = np.random.randn(5, 5+3)
bf = np.random.randn(5,1)
Wi = np.random.randn(5, 5+3)
bi = np.random.randn(5,1)
Wo = np.random.randn(5, 5+3)
bo = np.random.randn(5,1)
Wc = np.random.randn(5, 5+3)
bc = np.random.randn(5,1)
Wy = np.random.randn(2,5)
by = np.random.randn(2,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a_next, c_next, yt, cache = lstm_cell_forward(xt, a_prev, c_prev, parameters)
# print("a_next[4] = ", a_next[4])
# print("a_next.shape = ", c_next.shape)
# print("c_next[2] = ", c_next[2])
# print("c_next.shape = ", c_next.shape)
# print("yt[1] =", yt[1])
# print("yt.shape = ", yt.shape)
# print("cache[1][3] =", cache[1][3])
# print("len(cache) = ", len(cache))

In [448]:
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, a0, parameters):
    """
    Implement the forward propagation of the recurrent neural network using an LSTM-cell described in Figure (3).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    parameters -- python dictionary containing:
                        Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                        Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                        Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                        Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    # Initialize "caches", which will track the list of all the caches
    caches = []
    
    ### START CODE HERE ###
    # Retrieve dimensions from shapes of x and Wy (≈2 lines)
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wy"].shape
    
    # initialize "a", "c" and "y" with zeros (≈3 lines)
    a = np.zeros((n_a, m, T_x))
    c = np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # Initialize a_next and c_next (≈2 lines)
    a_next = a0
    c_next = np.zeros(a_next.shape)
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, next memory state, compute the prediction, get the cache (≈1 line)
        a_next, c_next, yt, cache = lstm_cell_forward(x[:, :, t], a_next, c_next, parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # print("a = ", a)
        # Save the value of the prediction in y (≈1 line)
        y[:,:,t] = yt
        print("yt = ", yt)
        # Save the value of the next cell state (≈1 line)
        c[:,:,t]  = c_next
        # print("c = ", c)
        # Append the cache into caches (≈1 line)
        caches.append(cache)
        
    ### END CODE HERE ###
    
    # store values needed for backward propagation in cache
    caches = (caches, x)

    return a, y, c, caches

In [449]:
np.random.seed(1)
# x = np.random.randn(3,10,7)

x = np.array([
 [[0, 0.475429975]],
 [[0.475429975, 1]],
 [[1, 0.427518428]],
 [[0.427518428, 0.771498771]],
 ])
print(x.shape)

n_a =5
n_x = 4
col = 1
n_y = 2

a0 = np.random.randn(n_a,col)
Wf = np.random.randn(n_a, n_a + n_x)
bf = np.random.randn(n_a,1)
Wi = np.random.randn(n_a, n_a + n_x)
bi = np.random.randn(n_a,1)
Wo = np.random.randn(n_a, n_a + n_x)
bo = np.random.randn(n_a,1)
Wc = np.random.randn(n_a, n_a + n_x)
bc = np.random.randn(n_a,1)
Wy = np.random.randn(n_y,n_a)
by = np.random.randn(n_y,1)

parameters = {"Wf": Wf, "Wi": Wi, "Wo": Wo, "Wc": Wc, "Wy": Wy, "bf": bf, "bi": bi, "bo": bo, "bc": bc, "by": by}

a, y, c, caches = lstm_forward(x, a0, parameters)
print("a= ", a)
# print("a.shape = ", a.shape)
print("y =", y)
# print("y.shape = ", y.shape)
# print("caches[1][1[1]] =", caches[1][1][1])
# print("c[1][2][1]", c[1][2][1])
# print("len(caches) = ", len(caches))

(4, 1, 2)
yt =  [[0.82298573]
 [0.17701427]]
yt =  [[0.63447059]
 [0.36552941]]
a=  [[[-0.42654738 -0.34469274]]

 [[ 0.45849613  0.00785664]]

 [[-0.72340109 -0.06489487]]

 [[ 0.48066157  0.83271167]]

 [[-0.11921914  0.03771391]]]
y = [[[0.82298573 0.63447059]]

 [[0.17701427 0.36552941]]]


In [450]:
for arr in x:
    print(", ".join(str(x) for x in arr))

[0.         0.47542998]
[0.47542998 1.        ]
[1.         0.42751843]
[0.42751843 0.77149877]


In [451]:
for arr in a0:
    print(", ".join(str(x) for x in arr))

1.6243453636632417
-0.6117564136500754
-0.5281717522634557
-1.0729686221561705
0.8654076293246785


In [452]:
for arr in Wf:
    print(", ".join(str(x) for x in arr))

-2.3015386968802827, 1.74481176421648, -0.7612069008951028, 0.31903909605709857, -0.2493703754774101, 1.462107937044974, -2.060140709497654, -0.3224172040135075, -0.38405435466841564
1.1337694423354374, -1.0998912673140309, -0.17242820755043575, -0.8778584179213718, 0.04221374671559283, 0.5828152137158222, -1.1006191772129212, 1.1447237098396141, 0.9015907205927955
0.5024943389018682, 0.9008559492644118, -0.6837278591743331, -0.12289022551864817, -0.9357694342590688, -0.2678880796260159, 0.530355466738186, -0.691660751725309, -0.39675352685597737
-0.6871727001195994, -0.8452056414987196, -0.671246130836819, -0.01266459891890136, -1.1173103486352778, 0.23441569781709215, 1.6598021771098705, 0.7420441605773356, -0.19183555236161492
-0.8876289640848363, -0.7471582937508376, 1.6924546010277466, 0.05080775477602897, -0.6369956465693534, 0.19091548466746602, 2.100255136478842, 0.12015895248162915, 0.6172031097074192


In [453]:
for arr in bf:
    print(", ".join(str(x) for x in arr))

0.3001703199558275
-0.35224984649351865
-1.1425181980221402
-0.3493427224128775
-0.2088942333747781


In [454]:
for arr in Wi:
    print(", ".join(str(x) for x in arr))

0.5866231911821976, 0.8389834138745049, 0.9311020813035573, 0.2855873252542588, 0.8851411642707281, -0.7543979409966528, 1.2528681552332879, 0.5129298204180088, -0.29809283510271567
0.48851814653749703, -0.07557171302105573, 1.131629387451427, 1.5198168164221988, 2.1855754065331614, -1.3964963354881377, -1.4441138054295894, -0.5044658629464512, 0.16003706944783047
0.8761689211162249, 0.31563494724160523, -2.022201215824003, -0.3062040126283718, 0.8279746426072462, 0.2300947353643834, 0.7620111803120247, -0.22232814261035927, -0.20075806892999745
0.1865613909882843, 0.4100516472082563, 0.19829972012676975, 0.11900864580745882, -0.6706622862890306, 0.3775637863209194, 0.12182127099143693, 1.1294839079119197, 1.198917879901507
0.18515641748394385, -0.3752849500901142, -0.6387304074542224, 0.4234943540641129, 0.07734006834855942, -0.3438536755710756, 0.04359685683424694, -0.6200008439481293, 0.6980320340722189


In [455]:
for arr in bi:
    print(", ".join(str(x) for x in arr))

-0.4471285647859982
1.2245077048054989
0.4034916417908
0.593578523237067
-1.0949118457410418


In [456]:
for arr in Wo:
    print(", ".join(str(x) for x in arr))

0.1693824330586681, 0.7405564510962748, -0.9537006018079346, -0.26621850600362207, 0.03261454669335856, -1.3731173202467557, 0.31515939204229176, 0.8461606475850334, -0.8595159408319863
0.35054597866410736, -1.3122834112374318, -0.038695509266051115, -1.6157723547032947, 1.121417708235664, 0.4089005379368278, -0.024616955875778355, -0.7751616191691596, 1.2737559301587766
1.9671017492547347, -1.857981864446752, 1.2361640304528203, 1.6276507531489064, 0.3380116965744758, -1.199268032335186, 0.8633453175440216, -0.18092030207815046, -0.6039206277932573
-1.2300581356669618, 0.5505374959762154, 0.7928068659193477, -0.6235307296797916, 0.5205763370733708, -1.1443413896231427, 0.8018610318713447, 0.04656729842414554, -0.18656977190734877
-0.10174587252914521, 0.8688861570058679, 0.7504116398650081, 0.5294653243527092, 0.13770120999738608, 0.07782112791270591, 0.6183802619985245, 0.23249455917873788, 0.6825514068644851


In [457]:
for arr in bo:
    print(", ".join(str(x) for x in arr))

-0.31011677351806
-2.434837764107139
1.038824601859414
2.1869796469742577
0.44136444356858207


In [458]:
for arr in Wc:
    print(", ".join(str(x) for x in arr))

-0.10015523328349978, -0.13644474389603303, -0.11905418777480989, 0.0174094083000046, -1.1220187287468883, -0.5170944579202279, -0.997026827650263, 0.2487991613877705, -0.29664115237086275
0.4952113239779604, -0.17470315974250095, 0.986335187821242, 0.21353390133544178, 2.1906997289697334, -1.8963609228910925, -0.646916688254908, 0.901486891648711, 2.528325706806398
-0.24863477771546005, 0.043668993178389105, -0.22631424251360518, 1.3314571125875918, -0.2873078634760189, 0.6800698398781045, -0.3198015988986712, -1.2725587552459943, 0.31354772046343216
0.5031848134353261, 1.2932258825322618, -0.11044702641731631, -0.6173620637123609, 0.5627610966190263, 0.24073709223773224, 0.28066507712263905, -0.07311270374727777, 1.1603385699937696
0.36949271637572373, 1.9046587083409812, 1.1110566985605046, 0.6590497961002102, -1.6274383406162574, 0.602319280295629, 0.42028220364705954, 0.8109516728035557, 1.0444420947072588


In [459]:
for arr in bc:
    print(", ".join(str(x) for x in arr))

-0.40087819178892664
0.8240056184504077
-0.5623054310190898
1.9548780750090344
-1.3319516665172482


In [460]:
for arr in Wy:
    print(", ".join(str(x) for x in arr))

-1.7606885603987834, -1.6507212658241002, -0.8905555841630485, -1.119115398559728, 1.956078903703642
-0.32649949807818424, -1.342675789377436, 1.114382976779792, -0.5865239388215925, -1.2368533765413974


In [461]:
for arr in by:
    print(", ".join(str(x) for x in arr))

0.8758389276492995
0.6233621765780327
